In [1]:
# exploring data attached 
import pandas as pd 


In [2]:
jobs = pd.read_excel("GA_task.xlsx")
jobs

,1,Unnamed: 1,2,Unnamed: 3,3,Unnamed: 5,4,Unnamed: 7,5,Unnamed: 9,...,46,Unnamed: 91,47,Unnamed: 93,48,Unnamed: 95,49,Unnamed: 97,50,Unnamed: 99
0,R,T,R,T,R,T,R,T,R,T,...,R,T,R,T,R,T,R,T,R,T
1,9,22,5,17,5,24,8,40,9,14,...,7,21,9,24,6,37,9,26,10,36
2,3,49,10,13,4,28,5,47,7,45,...,3,10,10,40,7,22,1,39,2,46
3,1,47,7,23,1,18,5,47,6,30,...,8,22,9,24,3,39,6,31,4,16
4,9,30,4,29,5,32,7,18,2,20,...,7,27,7,22,2,49,10,13,5,18
5,5,21,6,27,4,17,7,27,10,42,...,7,50,4,26,4,19,1,20,2,21
6,3,19,8,33,10,31,2,43,1,29,...,10,29,8,17,3,38,10,21,2,33
7,2,18,2,49,8,50,7,33,4,21,...,10,33,7,12,6,24,5,25,8,46
8,7,45,9,39,7,34,1,10,9,35,...,10,11,6,42,8,20,4,18,3,40
9,6,10,1,15,5,38,1,35,7,43,...,5,13,2,24,4,48,6,40,7,27


So to not get to confused with vocabulary. We face following problem we have 50 `tasks` each of them requires Resource R for some period of time T. Let's call one single element of this Task a Job for each Task Jobs need to be performed within declared order. Each resource we'll call a Machine so for job to be done we need some working time on specified machine.

In [3]:

class Task: 
    def __init__(self , jobs: list , id:int):
        """
        Represents a single task that requries some jobs within order to be done 
        Jobs a list of pair R - T for this task 
        """
        self.jobs = jobs                 # Jobs in order to be performed
        self.id = id            # as a Id in excel file is 1 -index and python is  0 - index
        self.current_timestamp = 0     # in our timeline kinda time mark at which we're with task so for example if we start first job at time 0 and 15 T is required we set this to 15.                        
    def __str__(self) -> str:
        return f"Task id: {self.id} \n jobs to perform: \n {self.jobs}"
    
    def return_machines(self):
        machines_required = []
        for job in self.jobs:
            machines_required.append(job[0])
        return machines_required
    
    def get_timestamp(self):
        return self.current_timestamp
    
    def get_machine_ix(self):
        return  self.jobs[0][0] - 1       # first job -> resource (1 index) - 1 to get 0_ix 
    
    def get_T(self):
        return  self.jobs[0][1]
    
    def next_step(self,time):              # we update our task after scheduling of first job to do
        self.jobs.pop(0)
        self.current_timestamp = time
    
   
        

In [4]:
class Machine:
    """Initialize machine object """
    def __init__(self, ix):
        self.id = ix   
        self.busy_until = 0   
        self.schedule =  []  # for now let's separete self.tasks for plot or even omit and create another parameter with let's call it occupation slots start-end of job performed on machine
        self.tasks = [[],[],[],[]]  # we'll try to perform visualization in a gant chart style, probably will work only on some small subset of our tasks 
        # format to then convert it to dataframe and utilize plotly so [[machine_id],[task_id],[start_time],[end_time]
        self.schedule.append((0,0)) # for consistency let's assume that on start machine is occupied with start at 0 and end at 0
        # self.task_scheduled = 1  # we now that at init task list has one trivial task
    def get_timestamp(self):
        return self.busy_until
        
    # let's also here add method to give a machine abilty to return slot at which it can perform given task, 
    def schedule_task(self,timestamp,time):
        """Timestamp first point in time at which task can be scheduled, time -> for task big T - time to perform job"""
        # we need to keep schedule in order 
        length = len(self.schedule)
        if length == 1:
            self.schedule.append((timestamp,timestamp+time))
            self.busy_until = timestamp+time
            self.tasks[2].append(timestamp)                        # getting our table for chart ready probably to improve
            self.tasks[3].append(timestamp+time)
            self.tasks[0].append(self.id)
            # self.task_scheduled +=1
            return timestamp+time
        else:
            # self.task_sch
            # eduled +=1
            for i in range(1,length):
                end_of_previous_job = self.schedule[i-1][1]
                start_of_next_job = self.schedule[i][0]
             
                if ((timestamp <= end_of_previous_job) & (end_of_previous_job+time <=start_of_next_job)):   # checking two conditions to answer question Can we fit new task somewhere between old scheduled tasks
                    self.schedule.insert(i,(end_of_previous_job,end_of_previous_job+time))   # put task into gap
                    
                    self.tasks[2].append(end_of_previous_job)
                    self.tasks[3].append(end_of_previous_job+time)
                    # only situation that end of current job in the task will differ from machine.busy_unitl
                    self.tasks[0].append(self.id)
                    # print(timestamp+time)
                    return end_of_previous_job+time # we need this to update task timeline 
                # polish cuz tired -> tu opisaliśmy sytuację gdzie możemy zacząć zadanie w trakcie kiedy jeszcze wyonujemy coś na maszynie, ale zmiescimy zadanie przed kolejnym już zaplanowanym 
                # mozemy jeszcze mieć sytuację, ze zaczniemy zadanie w trakcie przerwy, ale będizemy mogli rozpocząć dopiero po jakimś czasie od zwolnienia maszyny, ale i tak się wyrabiamy przed kolejnym zaplanowanym
                elif((end_of_previous_job <= timestamp <= start_of_next_job) &  (timestamp+time <=start_of_next_job)):
                    
                    self.schedule.insert(i,(timestamp,timestamp+time))   # put task into gap
                    self.tasks[2].append(timestamp)
                    self.tasks[3].append(timestamp+time)
                    # only situation that end of current job in the task will differ from machine.busy_unitl
                    self.tasks[0].append(self.id)
                    # print(timestamp+time)
                    return timestamp+time # we need this to update task timeline 
                
                
                    
            else:
                # we need to append at the end of task list for the machine but also first check if task.timepoint (end of previous job) > machine.timepoint(end of last scheduled task)
                if (timestamp<self.busy_until):
                    self.schedule.append((self.busy_until,self.busy_until+time))
                    
                    self.tasks[2].append(self.busy_until)
                    self.tasks[3].append(self.busy_until+time)
                    
                    self.busy_until += time
                    self.tasks[0].append(self.id)
                    return self.busy_until
                else:
                    self.schedule.append((timestamp,timestamp+time))
                    self.busy_until = timestamp + time
                    self.tasks[2].append(timestamp)
                    self.tasks[3].append(timestamp+time)
                    self.tasks[0].append(self.id)
                    return timestamp+time
        
        # print(self.busy_until)
        # return self.busy_until # in every other situation we can return self.busy_until

    def update_task_list(self,task_id):
        # during step of adding we also add task_id to the table:
        self.tasks[1].append(task_id+1) 
        
    def df_task_list(self):
        return pd.DataFrame({'machine_id':self.tasks[0],'task_id':self.tasks[1],'start_time':self.tasks[2],'end_time':self.tasks[3]})
    
    # def next_step(self):
    #     self.task_scheduled += 1
        
        
        

In [118]:
import random
import copy
from collections import Counter

def read_tasks(n = None):
    jobs = pd.read_excel("GA_task.xlsx")
    table = []
    col_number = len(jobs.columns)
  
    if n:
        length = 0
        for i in range(0, col_number,2):
            id = jobs.columns[i]
            task_jobs = list(zip(jobs.iloc[1:,i],jobs.iloc[1:,i+1]))
            table.append(Task(jobs=task_jobs,id = id)) # we add zipped pairs of machine and time for each task , and we convert to 0 index
            length +=1
            if length == n: return table
        
    for i in range(0, col_number,2):
        id = jobs.columns[i]
        task_jobs = list(zip(jobs.iloc[1:,i],jobs.iloc[1:,i+1]))
        table.append(Task(jobs=task_jobs,id = id)) # we add zipped pairs of machine and time for each task , and we convert to 0 index
    
    return table


def extract_machines(task_table: list):
    """ Requires task_table -> list of Task objects"""
    list_of_machines = []
    for task in task_table:
        machines_for_task = task.return_machines()   # this method returns list of machines for this task
        for machine in machines_for_task:
            list_of_machines.append(machine)   # we append machine requried
    # in the end remove duplicates and return 
    return list(set(list_of_machines))
    
    
def shuffle(input_list, count):
    '''Shuffles any n number of values in a list'''
    indices_to_shuffle = random.sample(range(len(input_list)), k=count)
    to_shuffle = [input_list[i] for i in indices_to_shuffle]
    random.shuffle(to_shuffle)
    for index, value in enumerate(to_shuffle):
        old_index = indices_to_shuffle[index]
        input_list[old_index] = value
    return input_list

# function to cross two genotypes # for now just simple crossing second half becoems second half of second genotype
def crossover(geno1, geno2,len_crossover):
    # we can set a len of crossover 
    #Order crossover (OX1)
    geno1 = copy.deepcopy(geno1)   
    geno2 = copy.deepcopy(geno2)  
    
    perm1 = geno1.geno #  1st permutation 
    perm2 = geno2.geno # 2nd permuation
    size = len(perm1)
    child1, child2 = [None]*size, [None]*size
   
    # Step 1: Choose two random crossover points
    cx_point1, cx_point2 = sorted(random.sample(range(size), 2))

    # Step 2: Copy the segment between cx_point1 and cx_point2
    child1[cx_point1:cx_point2] = perm1[cx_point1:cx_point2]
    child2[cx_point1:cx_point2] = perm1[cx_point1:cx_point2]
    
    # Track occurrences of elements already placed in children
    counter_child1 = Counter(child1[cx_point1:cx_point2])
    counter_child2 = Counter(child2[cx_point1:cx_point2])

    # Step 3: Fill in the remaining positions from the other parent, skipping duplicates
    def fill_remaining_genes(child, other_parent, start, end, counter_child):
        current_pos = end
        counter_parent = Counter(other_parent)
        
        for gene in other_parent:
            if counter_child[gene] < counter_parent[gene]:
                while child[current_pos] is not None:
                    current_pos = (current_pos + 1) % size
                child[current_pos] = gene
                counter_child[gene] += 1
    # Create the two children
    geno1.geno = fill_remaining_genes(child1, perm2, cx_point1, cx_point2,counter_child1)
    geno2.geno = fill_remaining_genes(child2, perm1, cx_point1, cx_point2,counter_child2)

    return geno1, geno2
    

    
    

    
    
    

In [119]:
genz , genx = Genotype(task_table=task_table,machine_table=machine_table,n_jobs=11,seed= 420) , Genotype(task_table=task_table,machine_table=machine_table,n_jobs=11,seed= 42)

In [120]:
geny , gena = crossover(genx,genz , len_crossover=100 )
geny.geno , gena.geno 

(None, None)

In [6]:
task_table = read_tasks()
machines = extract_machines(task_table)
machine_table = []

# now we are ready to initialize table of machines 

for ix in machines:
    machine_table.append(Machine(ix = ix))

In [7]:
# and now finally Genotype here we'll probably make the most of calcualtion and also for each genotype we need to calcualte time to complete tasks in given order 
# so to do that we'll utilize Machine and Task classes and their methods
import random
import copy
class Genotype:
    def __init__(self,task_table,machine_table,n_jobs=11,seed=None):
        """ It needs to be initialized with tables of Tasks and Machines and number of jobs in task"""
        # with assumption that each task requires same number of jobs length of genotype will be len(task_table) * len(machine_table)
        if seed is not None:
            random.seed(seed)
        self.geno = self.initialize_geno(len(task_table),n_jobs)
        self.task_table = copy.deepcopy(task_table)
        self.machine_table = copy.deepcopy(machine_table)  # to avoid manuplating original input and then reinitializing it and so on 
        self.time = 0 
      
    def initialize_geno(self,n,k):
        geno = list(range(0,n)) * k     # just creating random genotype 
        random.shuffle(geno)
        return geno
        
    def calculate_time(self): 
        for task_ix in self.geno:
            machine_ix = self.task_table[task_ix].get_machine_ix()  # extracting first machine_x from task
            task_timestamp = self.task_table[task_ix].get_timestamp()   # extracting current position in time of this task end
            # machine_timestamp = self.machine_table[machine_ix].get_timestamp()  # extracting current position in time of machine required
            # # two options
            time = self.task_table[task_ix].get_T() # getting Time of job
            
            task_till = self.machine_table[machine_ix].schedule_task(task_timestamp,time)   # this function will return point in time at which we're with this task
            # just for future visualization
            self.machine_table[machine_ix].update_task_list(task_ix)
            # and now we need to pop this job from task requirments It's done or at least planned to be done
            self.task_table[task_ix].next_step(task_till)   # we remove first job from job list for this task and we update point in time of this task
            
            # print(f"Task_id: {task_ix}, Machine: {machine_ix} , zadanioe zaplanowane do: {task_till}, zadanie może się rozpocząć: {task_timestamp}, maszyna zajęta do {self.machine_table[machine_ix].busy_until}")
            # self.machine_table[machine_ix].next_step()   # just keep tracking of number scheduled jobs for machine
             
        # at the end of this loop we have all machines in table scheduled we now we can extract time at which each of them finish and max of it is time in this genotype
        
        max = 0
        for machine in self.machine_table:
            if (machine.busy_until > max):
                max = machine.busy_until
        self.time = max
        return self.time
    
    def mutate(self,percent = 10):
        """Function that we call to mutate genotype (random shuffle some part of it)

        Args:
            percent (int) percent of genotype we want to shuffle for example 20 means shuffle 1/5 elements of geno. In algorithm we'll have one more variable because 
            we''ll be able to decide how many genotype mutate
        """
        
        n_to_shuffle = len(self.geno) // percent
        
        self.geno = shuffle(self.geno,n_to_shuffle)
        
        
        
    
    
    
    def reset(self,task_table,machine_table):   # it doesnt reset geno
        # self.geno = self.initialize_geno(len(task_table),n_jobs)
        self.task_table = copy.deepcopy(task_table)
        self.machine_table = copy.deepcopy(machine_table)  # to avoid manuplating original input and then reinitializing it and so on 
        self.time = 0 
                
        
        
            
            
        
        
        
        
        


In [ ]:
# genetic algorithm 
#population -> mutation -> crossover -> genotype to fenotype treansition that's our interprantation of schedule and then calcualte time as a fitness function
# -> probability of survival -> selection 
import copy

def genethic_algorithm(population_size ,  task_table , machine_table, survival_number = 4, steps_number = None ,mutation_percent= None , percent_to_cross = 50):
    
    # initialize population 
    
    population = [Genotype(task_table,machine_table,n_jobs=11) for _ in range(population_size)]
    
    gen_length = len(population[0].geno)
    
    # calculate fitness 
    
    fitness_table = [(population[ix],population[ix].calculate_time()) for ix in range(population_size)]#.sort(key=lambda x: x[1])
    
    #fitness_table = fitness_table.sort(key=lambda x: x[1])
    fitness_table.sort(key=lambda x: x[1])  # sorting by ascending time
    
    print(f"Step 0: best time = {fitness_table[0][1]} \n")

    # we'll always keep at least some part of survival without any changes but make sure to mutate at least some of surivals
    for _ in range(1,steps_number):
        
        new_population = [x[0] for x in fitness_table[:survival_number]]   # tu mamy 4 najlepsze 
        
        
        
        for item in new_population:
            item.reset(task_table,machine_table)
        
    
        # mutacje 
        
        for genotype in range(survival_number):    # mutatcja
            mutation = copy.deepcopy(new_population[genotype])
            mutation.reset(task_table,machine_table)
            mutation.mutate(percent=mutation_percent)
            new_population.append(mutation)
            
        # let's add crossovers 
        

        
        
        for i in range(1,survival_number,2):
            
            cross1, cross2 = crossover(new_population[i-1],new_population[i],elements=percent_to_cross * gen_length // 100)
            # cross1.reset(task_table=task_table,machine_table=machine_table)
            # cross2.reset(task_table=task_table,machine_table=machine_table)
            new_population.append(cross1)
            new_population.append(cross2)
       
        
            
        # and append 2 more random elements 
        
        new_population.append(Genotype(task_table=task_table,machine_table=machine_table,n_jobs=11))
        new_population.append(Genotype(task_table=task_table,machine_table=machine_table,n_jobs=11))
        
        # return new_population ,2
        size = len(new_population)
        # return new_population[8],2
        fitness_table = [(new_population[ix],new_population[ix].calculate_time()) for ix in range(size)]#.sort(key=lambda x: x[1])
        fitness_table = fitness_table.sort(key=lambda x: x[1])
        fitness_table.sort(key=lambda x: x[1])  # sorting by ascending tim
               
        
        print(f"Step {_}: best time = {fitness_table[0][1]} \n")
            
        # errors 
    
    # at the end let's return top 5 of fitness table
    
    return fitness_table[:5] , new_population 
    
    

In [71]:
top_5_table , aa = genethic_algorithm(15,task_table=task_table,machine_table=machine_table,mutation_percent=10,percent_to_cross=50,steps_number=10)

Step 0: best time = 2148 



In [84]:
top_5_table.geno

[42,
 9,
 21,
 26,
 29,
 33,
 37,
 41,
 48,
 34,
 42,
 13,
 49,
 11,
 11,
 47,
 25,
 21,
 42,
 20,
 40,
 7,
 18,
 47,
 24,
 28,
 35,
 25,
 28,
 31,
 48,
 38,
 34,
 46,
 42,
 12,
 2,
 4,
 48,
 44,
 38,
 22,
 39,
 49,
 38,
 31,
 36,
 17,
 37,
 5,
 25,
 15,
 12,
 44,
 10,
 3,
 11,
 27,
 40,
 48,
 48,
 37,
 46,
 23,
 28,
 43,
 23,
 39,
 17,
 16,
 40,
 25,
 35,
 14,
 46,
 7,
 18,
 21,
 21,
 13,
 14,
 21,
 12,
 20,
 42,
 41,
 15,
 9,
 9,
 37,
 31,
 6,
 44,
 29,
 22,
 28,
 24,
 34,
 12,
 1,
 4,
 14,
 46,
 21,
 10,
 27,
 11,
 6,
 22,
 38,
 41,
 18,
 11,
 31,
 30,
 7,
 48,
 16,
 34,
 31,
 37,
 26,
 3,
 14,
 23,
 10,
 18,
 40,
 49,
 15,
 26,
 36,
 37,
 3,
 45,
 35,
 12,
 45,
 35,
 28,
 16,
 22,
 17,
 47,
 23,
 27,
 13,
 23,
 3,
 26,
 37,
 2,
 22,
 27,
 36,
 49,
 7,
 30,
 23,
 21,
 5,
 5,
 9,
 47,
 26,
 20,
 43,
 4,
 30,
 38,
 12,
 9,
 40,
 18,
 10,
 49,
 42,
 24,
 45,
 26,
 8,
 26,
 10,
 8,
 4,
 20,
 21,
 45,
 47,
 21,
 13,
 14,
 39,
 7,
 39,
 3,
 24,
 2,
 22,
 44,
 33,
 13,
 16,
 16,
 48,
 46,


In [79]:
for i in top_5_table.task_table:
    print(i)

Task id: 1 
 jobs to perform: 
 [(9, 22), (3, 49), (1, 47), (9, 30), (5, 21), (3, 19), (2, 18), (7, 45), (6, 10), (8, 39), (5, 46)]
Task id: 2 
 jobs to perform: 
 [(5, 17), (10, 13), (7, 23), (4, 29), (6, 27), (8, 33), (2, 49), (9, 39), (1, 15), (9, 12), (7, 13)]
Task id: 3 
 jobs to perform: 
 [(5, 24), (4, 28), (1, 18), (5, 32), (4, 17), (10, 31), (8, 50), (7, 34), (5, 38), (1, 29), (1, 26)]
Task id: 4 
 jobs to perform: 
 [(8, 40), (5, 47), (5, 47), (7, 18), (7, 27), (2, 43), (7, 33), (1, 10), (1, 35), (7, 15), (9, 39)]
Task id: 5 
 jobs to perform: 
 [(9, 14), (7, 45), (6, 30), (2, 20), (10, 42), (1, 29), (4, 21), (9, 35), (7, 43), (1, 12), (7, 36)]
Task id: 6 
 jobs to perform: 
 [(6, 33), (3, 50), (9, 19), (3, 28), (3, 25), (5, 26), (7, 30), (5, 34), (5, 15), (7, 12), (9, 35)]
Task id: 7 
 jobs to perform: 
 [(10, 48), (4, 17), (7, 40), (6, 44), (8, 35), (6, 16), (3, 47), (7, 43), (8, 47), (3, 17), (4, 17)]
Task id: 8 
 jobs to perform: 
 [(3, 32), (3, 36), (1, 36), (2, 21), (10

In [72]:
top_5_table.time

0

In [39]:
aa

2151

In [19]:
for geno in top_5_table:
    print(geno.time)

0
0
0
0
0
0
0
0
0
0
0
0
0
0


In [ ]:
testowy_genotyp = Genotype(task_table=task_table,machine_table=machine_table)

In [ ]:
testowy_genotyp.calculate_time()

2220

In [ ]:
from plotly.express import bar
def visualize_schedule(machine_table,total_time):
    """ Requires machine table (schedule) for specific genotype"""
    df_list = []
    for i in machine_table:
        df_list.append(i.df_task_list())
    df_sumup = pd.concat(df_list,axis=0)
    df_sumup['end'] = df_sumup['end_time'] - df_sumup['start_time']
    fig = bar(data_frame=df_sumup,base='start_time',x = 'end',y = 'machine_id',color = "task_id",orientation='h',title=f"Total time: {total_time}")
    fig.show()
visualize_schedule(testowy_genotyp.machine_table,total_time)